In [45]:
import pycrfsuite
import os,json,logging
logging.basicConfig(level=logging.INFO)

In [46]:
def _word2features(wordlist, i):
    """ 返回特征列表 """
    features = [
        'bias',
        'word:'+wordlist[i][0],
        'word_attr:'+wordlist[i][1]
    ]

    if i > 0:
        features.append('word[-1]:'+wordlist[i-1][0])
        features.append('word[-1]_attr:'+wordlist[i-1][1])
        if i > 1:
            features.append('word[-2]:'+wordlist[i-2][0])
            features.append('word[-2,-1]:'+wordlist[i-2][0]+wordlist[i-1][0])
            features.append('word[-2]_attr:'+wordlist[i-2][1])
    if i < len(wordlist)-1:
        features.append('word[1]:'+wordlist[i+1][0])
        features.append('word[1]_attr:'+wordlist[i+1][1])
        if i < len(wordlist)-2:
            features.append('word[2]:'+wordlist[i+2][0])
            features.append('word[1,2]:'+wordlist[i+1][0]+wordlist[i+2][0])
            features.append('word[2]_attr:'+wordlist[i+2][1])
    return features

In [66]:
def _fragment(inlist, allist, token):
    pos0=0
    for i in range(len(inlist)):
        if inlist[i][1]=='x' and inlist[i][0] in token:
            pos1=i+1
#             logging.info('%i ~ %i'%(pos0, pos1))
            allist.append(inlist[pos0:pos1])
            pos0=pos1
            
    return        

In [50]:
src = "seg_sym"

尝试两种句读.

第一种, 仅将句号都视作断句.

In [82]:
wlist1=[]
for _, _, fnames in os.walk("json/"+src):
    for fname in fnames:
        if '虔门' in fname: continue
        with open("json/"+src+'/'+fname, 'r') as f:
            content=json.load(f)
            logging.info('%s : %i'%(fname, len(content)))
            _fragment(content, wlist1, "。")
            logging.info('ALL LIST : %i'%(len(wlist1)))

INFO:root:seg_sym_宁.json : 19316
INFO:root:ALL LIST : 497
INFO:root:seg_sym_阴阳学堂-夺碑之役.json : 21924
INFO:root:ALL LIST : 899
INFO:root:seg_sym_阴阳学堂外编.json : 8582
INFO:root:ALL LIST : 1193


In [83]:
def sent2features(sent):
    return [_word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
%time
train_x, train_y = [sent2features(s) for s in wlist1], [sent2labels(s) for s in wlist1]

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 14.8 µs


In [84]:
%time
model = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(train_x, train_y):
    model.append(xseq, yseq)

model.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier
    'feature.possible_states': True,
    # include transitions that are possible, but not observed
    'feature.possible_transitions': True,
    'feature.minfreq': 3
})

CPU times: user 10 µs, sys: 5 µs, total: 15 µs
Wall time: 22.6 µs


开始训练!

In [85]:
%timeit
model.train("./no_qmmf_juhao.crf")

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.8 µs
Feature generation
type: CRF1d
feature.minfreq: 3.000000
feature.possible_states: 1
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 16188
Seconds required: 11.228

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 15747.100353
Feature norm: 1.000000
Error norm: 30505.255778
Active features: 16160
Line search trials: 1
Line search step: 0.000014
Seconds required for this iteration: 0.823

***** Iteration #2 *****
Loss: 3758.706263
Feature norm: 1.796186
Error norm: 4635.358677
Active features: 15210
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.462

***** Iteration #3 *****
Loss: 3197.671797
Feature norm: 1.940814
Error norm: 3010.687662
Active features: 9232
Line sear

***** Iteration #41 *****
Loss: 408.352136
Feature norm: 19.750243
Error norm: 24.955702
Active features: 554
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.446

***** Iteration #42 *****
Loss: 407.218753
Feature norm: 19.870236
Error norm: 58.831309
Active features: 546
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.471

***** Iteration #43 *****
Loss: 405.692427
Feature norm: 19.907020
Error norm: 53.011046
Active features: 529
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.477

***** Iteration #44 *****
Loss: 404.130927
Feature norm: 20.074037
Error norm: 63.437797
Active features: 518
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.442

***** Iteration #45 *****
Loss: 402.520733
Feature norm: 20.098916
Error norm: 33.229472
Active features: 507
Line search trials: 1
Line search step: 1.000000
Seconds required for this ite

***** Iteration #82 *****
Loss: 381.349065
Feature norm: 22.833498
Error norm: 27.766067
Active features: 398
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.921

***** Iteration #83 *****
Loss: 381.158103
Feature norm: 22.837348
Error norm: 30.203434
Active features: 393
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.454

***** Iteration #84 *****
Loss: 380.992652
Feature norm: 22.957557
Error norm: 39.141933
Active features: 398
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.467

***** Iteration #85 *****
Loss: 380.447321
Feature norm: 22.958046
Error norm: 8.109946
Active features: 399
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.459

***** Iteration #86 *****
Loss: 380.297249
Feature norm: 23.011024
Error norm: 22.619344
Active features: 397
Line search trials: 1
Line search step: 1.000000
Seconds required for this iter

要出结果了

In [95]:
%%time
yyxt_f="json/seg_阴阳学堂.json"
yyxt_list=[]
with open(yyxt_f,'r') as f:
    content=json.load(f)
    _fragment(content, yyxt_list, "。")
    logging.info('ALL LIST : %i'%(len(yyxt_list)))

INFO:root:ALL LIST : 13692


CPU times: user 6.64 s, sys: 871 ms, total: 7.51 s
Wall time: 11.9 s


In [97]:
%%time
tagger = pycrfsuite.Tagger()
tagger.open('./no_qmmf_juhao.crf')
test_xs = [sent2features(s) for s in yyxt_list]
test_y=[]
for test_x in test_xs:
    testy = tagger.tag(test_x)
    test_y.append(testy)

CPU times: user 1min 28s, sys: 43.7 s, total: 2min 11s
Wall time: 2min 12s


In [113]:
s=0
ci_l=[]
for i in range(len(test_y)):
    ci=''
    for j in range(len(test_y[i])):
        if test_y[i][j]!='O':
            ci=ci+yyxt_list[i][j][0]
            if test_y[i][j]=='E':
                ci_l.append(ci)
                ci=''
ci_l=list(set(ci_l))
print(ci_l)

['暗羽手', '应天法门总司', '五色笔', '此槌状', '无天法门', '逆刃刀', '氏骆砚', '入色声', '封策镇', '咒牵带咒', '佛家法门场', '利用诀咒', '逆文碑阵', '天法门', '掬仙阵', '入封策', '入碑阵', '衡陵逆文碑', '桃木锥', '两法门', '十二法门阵', '吧骆砚', '伊尹杖', '衡陵逆文碑阵', '左耳耳门穴', '幻兵军队', '暗羽法门', '回应天法门', '幻工技阴阳家', '叫骆砚', '入八公', '擎仙荷', '阵', '时候', '戳戳骆砚', '噬骨虫', '逆文碑', '入梧桐林', '入内崖', '幻草阵', '入山碑', '使节棒', '使节杖']


## 结论
是标注不好的锅